<a href="https://colab.research.google.com/github/casangi/ngcasa/blob/master/docs/ngcasa_imaging.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imaging


Radio interferometry data analysis applications and algorithms may be assembled from CNGI and ngCASA building blocks. A user may choose to implement their own analysis scripts, use a pre-packaged task similar to those in current CASA or embed ngCASA and CNGI methods in a production pipeline DAG.

Note : The following examples represent preliminary design ideas that illustrate how ngCASA science applications may be assembled. The API and usage example scripts will change as the details are refined. The current focus is to evaluate whether the CNGI and ngCASA functional design and infrastructure adequately addresses algorithmic needs and how it may be best leveraged for scaleable high performance computing. Questions raised via this initial exercise will guide the design of [future prototypes](https://ngcasa.readthedocs.io/en/latest/ngcasa_development.html#List-of-Future-Prototypes), continued evaluation the chosen infrastructure, and the final function hierarchy and API definition. 

[Edit this notebook in colab](https://colab.research.google.com/github/casangi/ngcasa/blob/master/docs/ngcasa_flagging.ipynb)   : Please note that these examples are currently notional, and not yet runnable in the colab!
Only the [Imaging Prototype](https://github.com/casangi/ngcasa/tree/master/docs/prototypes) code and notebooks are currently runnable. 

The following example use cases relate to the different types of images listed in https://casa.nrao.edu/casadocs/casa-5.0.0/synthesis-imaging/image-definition


## Cube and Continuum (narrow-field, wide-field and joint-mosaic)
Example : Cube or Continuum imaging (nterms=1 and nterms>1) for narrow-field, wide-field and joint mosaic imaging, including visibility pre-processing for topo-lsrk conversion and automasking.  This is a pipeline imaging use-case.

In [0]:
#--------------------------------------------------------------------- Data Selection
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)


# -------------------------------------------------------------- Visibility Preprocessing
# topo->lsrk + channel binning + ephemeris sources
cngi.vis.regridspw(vis_dataset, impars)
# Apply flags for all future steps
cngi.vis.applyflags(vis_dataset)
# Phasecenter rotation
cngi.vis.rotateuvw(vis_dataset) # TBD : Here, or inside _make_grid ? 


#--------------------------------------------------------------------- Image Definition
# Construct an empty image set
img_dataset = cngi.dio.write_image(impars)
# Set image weighting scheme 
ngcasa.imaging.make_imaging_weight(img_dataset, weightpars)
# Define gridding convolution functions. Aterm, Wterm, JointMosaic are specified here.
ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars)

#--------------------------------------------------------------------- Make initial images
# Make PSF
ngcasa.imaging.make_psf(img_dataset, vis_dataset, gridpars)
# Make PB
ngcasa.imaging.make_pb(img_dataset,vis_dataset,gridpars)
# Make Residual image and normalize it
ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, gridpars, normpars)

#------------------------------------------------------------------ Iteration Control
# Initialize the mask
ngcasa.imaging.make_mask(img_dataset,maskpars)
# < Interactive Clean GUI >
# Check convergence criteria
iter_rec = ngcasa.imaging.is_converged(img_dataset, iterpars, None)


# Perform iterative reconstruction
while( iter_rec['stopcode']=='continue' ):
    # -----------------------------------------------------------------------Minor cycle
    exec_rec = ngcasa.imaging.deconvolve_point(img_dataset, decpars, iter_rec)
    #-----------------------------------------------------------------------------------
    
    # -----------------------------------------------------------------------Major cycle
    # Model prediction
    ngcasa.imaging.predict_modelvis_image(img_dataset, vis_dataset, normpars, gridpars)
    # Make residual image and normalize it.
    ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, gridpars,normpars)
    #-----------------------------------------------------------------------------------

    #------------------------------------------------------------------Iteration Control
    # Update the mask
    ngcasa.imaging.make_mask(img_dataset,maskpars)
    # < Interactive Clean GUI >
    # Check convergence criteria
    iter_rec = ngcasa.imaging.is_converged(img_dataset, iterpars, exec_rec)
    #-----------------------------------------------------------------------------------

    
#------------------------------------------------------------------    Restoration
# Restore the model image
ngcasa.imaging.restore_model(img_dataset)


To Do : The calculation of the gradient of the objective function should be separated from the gridding step. This will allow options other than Chi-Square to be used for the optimization algorithm.  This requires the insertion of an explicit "data-model" step in between the model_prediction and make_residual steps of the major cycle.

A similar change must be made below for make_sd_residual(). 

## Other imaging algorithms (multi-term, rm-synthesis)

Wide-band multi-term imaging with wideband pb-correction may be run by setting up a cube major cycle followed by deconvolve_multiterm.

Rotation-measure synthesis may be called by setting up a full-stokes cube major cycle, followed by a deconvolve_rotation_measure_clean

In both these cases, the deconvolution algorithm starts with an image cube, transforms the image into the sky model space (sparse basis), performs the deconvolution in that space, and transforms the model back to the cube in preparation for the next major cycle.   Wideband Primary beam correction (for Stokes I) is done simply by ensuring 'flatsky' or 'commonpb' normaliation at the end of the make_residual step. 

In [0]:
# Prepare the residual image cube : Normalize to a common pb. This implements widebandpbcor. 
ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, gridpars, normtype='commonpb')

# Start iterative deconvolution...

    # Run the deconvolver (input : residual and psf cubes, output : model cube)
    ngcasa.imaging.deconvolve_multiterm_clea(img_dataset)

    # Model prediction
    ngcasa.imaging.predict_modelvis_image(img_dataset, vis_dataset, normpars, gridpars)

    # Make residual image and normalize gain
    ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, gridpars,normtype='commonpb') 

To Do : Evaluate the option of an explicit cube_to_skymodel() and skymodel_to_cube() step that includes cube_to_taylor, stokes_to_corr, linear_mosaic, multi_field model conversions, and may be replaced by a user who wishes to implement any other convertor (such as for time-variability multi-term imaging, or conversion from frequency to lamda-square space for rm-synthesis).

## Interactive Clean

There are two parts to interactive image reconstruction.

(1) Mask drawing/viewing : Use a GUI to interactively draw a mask or to simply visualize the current mask. This may be used in conjunction with the ngcasa.imaging.make_mask() to view and/or edit the resulting region.

(2) Editing iteration control parameters at run-time : The same GUI used for mask visualization may be used to display and accept edited values for user-parameters. 

In the above example, this interactive step would reside in between 'Update Mask' and 'Check convergence criteria'. 

See https://gitlab.nrao.edu/rurvashi/interactive-imaging-with-casa6  for an example (using casa6) of how this may be achieved via a stand-alone call to a GUI in-between the major and minor cycles. Convergence history may also be displayed at this stage, allowing for an interactive user to decide if iteration-control parameters should change or not. 

## Linear Mosaics and Joint mosaics

Three options exist.

(1) The gridder allows for joint mosaic phase gradients to be applied to gridding convolution functions. No change to the code shown above. This is equivalent to mosweight=False in casa6.casatasks.tclean()

(2) ngcasa.imaging.linear_mosaic may be used to combine restored images from different pointings (or clusters of joint-mosaic pointings). This is a post-deconvolution step.

(3) Use cngi.image.linear_mosaic() to calculate a weighted sum of images from different pointing subsets, in-between the major and minor cycles. 
- This option has the advantage of allowing smaller image sizes for individual gridder calls.
- This implicitly implements 'mosweight=True' of casa6.casatasks.tclean() because each pointing (or subset of pointings) is gridded and normalized separately. 

TBD : Will we get (3) by using (1) but just choosing the partition axis to be along fields ? Almost, but no imsize reduction, and it will always be mosweight=False. 

Below is an implementation of (3). TBD : Can this be simplified ? 

In [0]:
## Image Reconstruction with Linear Mosaics before deconvolution.
## Major cycle runs separately for each pointing (or subset of pointings)
## Minor cycle runs on a joint image.

img_datasets={}

for imfield in list_of_fields:
    
    # Construct a selected vis dataset for one pointing (or subset of pointings)
    vis_dataset = cngi.dio.read_vis(visname, selpars)

    ### Code blocks from above for visibility preprocessing, image definition, make_psf, make_pb. 
    cngi.vis.regridspw(vis_dataset, impars)
    cngi.vis.applyflags(vis_dataset)
    cngi.vis.rotateuvw(vis_dataset) # TBD : Here, or inside _make_grid ? 
    img_dataset = cngi.dio.write_image(impars)
    ngcasa.imaging.make_imaging_weight(img_dataset, weightpars)
    ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars)
    ngcasa.imaging.make_psf(img_dataset, vis_dataset, gridpars)
    ngcasa.imaging.make_pb(img_dataset,vis_dataset,gridpars)    
    
    # Make Residual image and normalize it (normalizing per pointing => mosweight=True)
    ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, gridpars, normpars)
    
    # Accumulate image datasets for each pointing (or subset of pointings)
    img_datasets[imfield] =  img_dataset
    
# Do a linear mosaic to generate the image to send to the minor cycle
cngi.image.linear_mosaic(img_datasets, img_linmos)

# Setup iteration control and masks
ngcasa.imaging.make_mask(img_linmos,maskpars)
iter_rec = ngcasa.imaging.is_converged(img_linmos, iterpars, None)

# Perform iterative deconvolution
while( iter_rec['stopcode']=='continue' ):
    exec_rec = ngcasa.imaging.deconvolve_point(img_linmos, decpars, iter_rec)

    for imfield in list_of_fields:
        # Regrid the linmos model image onto subset model images
        cngi.image.regrid(img_linmos, img_datasets[imfield])
        # Model prediction
        ngcasa.imaging.predict_modelvis_image(img_datasets[imfield], vis_dataset, normpars, gridpars)
        # Make residual image and normalize it.
        ngcasa.imaging.make_residual_image(img_datasets[imfield],vis_dataset, gridpars,normpars)

    # Do a linear mosaic to generate the image to send to the minor cycle
    cngi.image.linear_mosaic(img_datasets, img_linmos)
    
    # Update iteration control and masks
    ngcasa.imaging.make_mask(img_dataset,maskpars)
    iter_rec = ngcasa.imaging.is_converged(img_dataset, iterpars, exec_rec)

    
#------------------------------------------------------------------    Restoration
# Restore the model image
ngcasa.imaging.restore_model(img_dataset)


## Multi-field Imaging

Purpose : To image outlier sources in separate small images in addition to the main large image. 

Use-Cases : 
- A bright outlier source far from the region of interest would cause the image size to greatly increase if imaged as part of a single image. This single large image would also likely be mostly empty. 
- A bright outlier source requires a different gridding or deconvolution algorithm from the main field, but must be part of the same reconstruction run. A continuum detection experiment at the center of the field would require nterms=1 with (say) multiscale clean, but a bright point source at the half-power level of the primary beam has strong spectral structure induced by the primary beam. In this case, there is no interest in flux accuracy of the bright outlier source, but it must be modeled and subtracted. A multi-term point-source deconvolution algorithm with the standard gridder may be used on the outlier source while a multi-scale nterms=1 imaging run is done on the main field. 

Algorithm Steps : 
  - (1) The same data are gridded onto multiple uv-grids to form a list of observed images and psfs. 
  -  (2) Each such image_field is deconvolved separately in the image domain.  
  -  (3) Iteration control must be merged across image_fields. The return dicts from ngcasa.imaging.has_converged() for each image_field must be merged before parameters are sent to the individual deconvolvers. 
  -  (4) Model images from all image_fields are reconciled to handle overlap regions.
  (In case of overlap, use the specified order of the input image_fields to indicate precedence and to blank out overlapping model image pixels for all but one image_field. Or, apply weights. )
  -  (5) Predict model visibilities separately for each image_field, adding to the model_data array in the vis_dataset.  

Steps 1,2,5 are done independently per image_field. 
Steps 3 and 4 implement the desired relation between these fields for iteration-control and in model-prediction. 

Below is an implementation. TBD : Can this be simplified ?

In [0]:
## Image Reconstruction with Multi-Field imaging
## Major cycle runs once, using models predicted from all image fields.
## Minor cycle runs separately on each image-field

# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)
# Visibility pre-processing : topo->lsrk + channel binning
cngi.vis.regridspw(vis_dataset, impars)
# Apply flags for all future steps
cngi.vis.applyflags(vis_dataset)

# Construct a list of empty image sets
img_datasets={}
for i in range(N_field):
    img_datasets[i] = cngi.dio.write_image(impars[i]) 

# Set image weighting scheme 
ngcasa.imaging.make_imaging_weight(img_datasets[0], weightpars)
# Define gridding convolution functions. Aterm, Wterm, JointMosaic are specified here.
ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars)

iter_recs={}
for i in range(N_field):
    # Make PSF
    ngcasa.imaging.make_psf(img_dataset[i], vis_dataset, gridpars)
    # Make PB
    ngcasa.imaging.make_pb(img_dataset[i],vis_dataset,gridpars)
    # Make Residual image and normalize it
    ngcasa.imaging.make_residual_image(img_dataset[i],vis_dataset, gridpars, normpars)

    # Initialize the mask and iteration control
    ngcasa.imaging.make_mask(img_dataset[i],maskpars)
    iter_recs[i] = ngcasa.imaging.is_converged(img_linmos, iterpars, None)


# Merge iteration control rules/parameters across fields
## Implement logic here to reconcile all iter_recs[] into a single iter_rec to apply to all fields. 
    
# Perform iterative reconstruction
while(iter_rec['stopcode']=='continue'):
    for i in range(N_field):
        ngcasa.imaging.deconvolve(img_dataset[i], decpars)

    # Implement code to handle overlapping regions in the model list. 
    # Use list ordering to pick only the first model and blank overlapping regions in other model images   

    ## Model prediction (incremental additions)
    for i in range(N_field):
        ngcasa.imaging.predict_modelvis_image(img_dataset[i], vis_dataset, normpars, gridpars, incremental=True)

    for i in range(N_field):
        ## Make residual image and normalize it.
        ngcasa.imaging.make_residual_image(img_dataset[i],vis_dataset, gridpars,normpars)

        # Update the mask
        ngcasa.imaging.make_mask(img_dataset[i],maskpars,interactive=T/F)
    
# Restore the model images
for i in range(N_field):
    ngcasa.imaging.restore_model(img_dataset[i])

## Single Dish Imaging with Deconvolution
Purpose : To remove the effect of the Single Dish effective beam from the observed images.

Algorithm Steps : 

In [0]:
## Open datasets and pre-process as needed

# Make the observed image
ngcasa.imaging.make_single_dish_residual(img_dataset)
# Make the PSF
ngcasa.imaging.make_single_dish_psf(img_dataset)
    
# Initialize the mask and iteration control
ngcasa.imaging.make_mask(img_dataset,maskpars)
iter_rec = ngcasa.imaging.is_converged(img_dataset, iterpars, None)

# Perform iterative reconstruction
while( iter_rec['stopcode']=='continue' ):
    # Minor Cycle
    exec_rec = ngcasa.imaging.deconvolve_multiterm clean(img_dataset, decpars, iter_rec) # MSClean or MSMFS.

    # Major Cycle 
    ngcasa.imaging.make_single_dish_residual(img_dataset)
    
    # Update the mask and iteration control
    ngcasa.imaging.make_mask(img_dataset,maskpars)
    iter_rec = ngcasa.imaging.is_converged(img_dataset, iterpars, exec_rec)

# Restore the model image
ngcasa.imaging.restore_model(img_dataset)

## This example does not use the sd_weight_image(). TBD : Update to use it

## Joint Single Dish and Interferometer Imaging

Purpose : To use constraints from both INT and SD datasets during a joint reconstruction. 

Algorithm Steps : 

(1) Construct Cube PSFs and Residual Images from INT and SD datasets separately.

(2) Apply cngi.image.feather() to merge them and produce a new img_set containing joint information.

(3) Minor cycle : ngcasa.imaging.deconvolve()

(4) Iteration control and masking : Same as interferometer imaging

(5) Major cycle
     - For INT, follow the same process as in the example above
     - For SD, Residual image = Observed image - { Model image (conv) SD PSF }
     Call cngi.image.feather() to merge the new residual images.
     

In [0]:
## Open datasets and pre-process as needed

# Make the observed SD image
ngcasa.imaging.make_single_dish_residual(sd_imset)
# Make the SD PSF
ngcasa.imaging.make_single_dish_psf(sd_imset)

# Make the INT PSF
ngcasa.imaging.make_psf(int_imset, vis_dataset, gridpars)
# Make the INT PB
ngcasa.imaging.make_pb(int_imset, vis_dataset,gridpars)
# Make INT Residual image and normalize it
ngcasa.imaging.make_residual_image(int_imset, vis_dataset, gridpars, normpars)

    
# Feather the SD and INT Cubes together ( PSF and Residual )
joint_imset = ngcasa.imaging.feather(sd_dataset, int_dataset, 'psf')
joint_imset = ngcasa.imaging.feather(sd_dataset, int_dataset, 'residual')
    
# Initialize the mask and iteration control
ngcasa.imaging.make_mask(joint_imset,maskpars)
iter_rec = ngcasa.imaging.is_converged(joint_imset, iterpars, None)

# Perform iterative reconstruction
while( iter_rec['stopcode']=='continue' ):
    # Minor Cycle
    exec_rec = ngcasa.imaging.deconvolve_multiterm(joint_imset, decpars, iter_rec) # MS-Clean or MSMFS

    # Copy/transfer the joint model from joint_imset to sd_imset and int_imset
    
    # Major Cycle for SD
    ngcasa.imaging.make_single_dish_residual(sd_imset)
    # Make INT Residual image and normalize it
    ngcasa.imaging.make_residual_image(int_imset, vis_dataset, gridpars, normpars)
    # Feather the SD and INT Cubes together ( PSF and Residual )
    joint_imset = ngcasa.imaging.feather(sd_dataset, int_dataset, 'residual')
    
    # Update the mask and iteration control
    ngcasa.imaging.make_mask(joint_imset,maskpars)
    iter_rec = ngcasa.imaging.is_converged(joint_imset, iterpars, exec_rec)

# Restore the model image
ngcasa.imaging.restore_model(joint_imset)


## Saving Model Visibilities

Model prediction and saving comes in several flavors. 
(1) From a component list
(2) From a model image
(3) uv-cont-fit

Model prediction prior to calibration will require the target array to be 'model' whereas simulation will require the target array to be 'data'. ngcasa.imaging.predict_modelvis_xxx() methods take a parameter to specify target array. 

(1) Calculate visibilities from flux-component lists. They may be observatory calibrator models, or the outputs of an imaging algorithm that produces component lists (e.g. ASP)

In [0]:
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

# Preprocess for freq-frame conversions
cngi.vis.regridspw(vis-dataset)  # Convert from topo to lsrk

# Predict model visibilities ( in lsrk frame )
ngcasa.imaging.predict_modelvis_component(vis_dataset, component_list)

# Undo frame conversions
cngi.vis.regridspw(vis_dataset)  # Convert from lsrk back to data frame (topo)

# Write to disk
cngi.dio.write_zarr(vis_dataset)
    

(2) Calculate visibilities from images, by degridding. The images may be observatory calibrator models or the output of image reconstrunction.

In [0]:
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

# Preprocess for freq-frame conversions
cngi.vis.regridspw(vis_dataset)  # Convert from topo to lsrk

# Set up de-grid options for PSterm, Aterm, Wterm, JointMosaic are specified here.
ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars)

# Predict model visibilities ( in lsrk frame )
ngcasa.imaging.predict_modelvis_image(vis_dataset, img_dataset)

# Undo frame conversions
cngi.vis.regridspw(vis_dataset)  # Convert from lsrk back to data frame (topo)

# Write to disk
cngi.dio.write_zarr(vis_dataset)


    
(3) Baseline-based continuum model fitting ( used in UV-continuum subtraction )

In [0]:
# Construct a selected vis dataset
vis_dataset = cngi.dio.read_vis(visname, selpars)

# Preprocess for freq-frame conversions and phase rotation to get the source at the phasecenter
cngi.vis.regridspw(vis_dataset)  # Convert from topo to lsrk
cngi.vis.rotateuvw(vis_dataset)  # Rotate to get source at observation phase center

# Do the uvcontfit : Writes the 'model' array in the XDS vis_dataset
cngi.vis.uvcontfit(vis_dataset) # Fit a continuum model per baseline (valid only for a point source at phasecenter)

# Undo frame conversions and rotations
cngi.vis.rotateuvw(vis_dataset)
cngi.vis.regridspw(vis_dataset)  # Convert from lsrk back to data frame (topo)

# Write to disk
cngi.dio.write_zarr(vis_dataset)

## UV-Continuum Subtraction

There are three ways of implementing uv-continuum subtraction. They follow the three options described above for model prediction and saving.  Insert a  cngi.vis.uvsub() step just prior to the final write to zarr. 

In [0]:
## Pick one of the model prediction methods from above.
vis_dataset = cngi.dio.read_vis(visname, selpars)
cngi.vis.regridspw(vis_dataset)  # Convert from topo to lsrk
cngi.vis.rotateuvw(vis_dataset)  # Rotate to get source at observatory phase center
cngi.vis.uvcontfit(vis_dataset) # Fit a continuum model per baseline (valid only for a point source at phasecenter)
cngi.vis.rotateuvw(vis_dataset)
cngi.vis.regridspw(vis_dataset) 

# Subtract the model from the data. Specify parameters to uvsub to pick which array names to use
cngi.vis.uvsub(vis_dataset)

# Write to disk
cngi.dio.write_zarr(vis_dataset)

## Self-calibration
Antenna gains may be solved-for using the current model, and applied prior to re-imaging a new residual image.

Note : This example is primarily to illustrate the flexibility of the ngCASA and CNGI library interface. In practice, for self-calibration, one may prefer to restart with a fresh model image after updating calibration. 

In [0]:
## Major Cycle of image reconstruction, with self-calibration

# Set up de-grid options as needed. 
ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars)

# Model prediction, using current values from the pointing dataset
# This fills in the 'MODEL' array in the vis_dataset
ngcasa.imaging.predict_modelvis_image(img_dataset, vis_dataset, pointing_cal_dataset, normpars, gridpars)

# Solve for antenna gains ()
cal_dataset = ngcasa.calibration.solve(vis_dataset)

# Apply gain solutions
ngcasa.calibration.apply(vis_dataset, cal_dataset)

# Make residual image using the self-calibrated visibilities. 
ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, pointing_cal_dataset, gridpars,normpars)
#-----------------------------------------------------------------------------------

## Pointing Self-Calibration

Pair a pointing-offset solver from the calibration module 
with model prediction via A-Projection de-gridding. 
Incorporate the solve step within the imaging major cycle such that update pointing solutions (in a cal_dataset) are applied during the subsequent residual gridding step. 

A key difference between pointing self-calibration and regular self-calibration is that the apply step is not a standalone operation on the visibility dataset, but one that is tighly integrated with the gridding step (by applying phase-gradients to gridding convolution functions).

TBD : Check algorithm with listing in publication...



In [0]:
## Major Cycle of image reconstruction, for pointing self-calibration

# Set up de-grid options for Aterm with pointing offsets read from a pointing dataset.
ngcasa.imaging.make_gridding_convolution_function(img_dataset, gridpars)

# Model prediction, using current values from the pointing dataset
# This fills in the 'MODEL' array in the vis_dataset
ngcasa.imaging.predict_modelvis_image(img_dataset, vis_dataset, pointing_cal_dataset, normpars, gridpars)

# Solve for pointing offsets
pointing_cal_dataset = ngcasa.calibration.solve_pointing(vis_dataset)

# Make residual image and normalize it.
ngcasa.imaging.make_residual_image(img_dataset,vis_dataset, pointing_cal_dataset, gridpars,normpars)
#-----------------------------------------------------------------------------------


